In [1]:
import logging
import yaml

import astropy.units as u
from astropy.time import Time
from astropy.coordinates import AltAz, ICRS, EarthLocation, Angle
import asyncio

import palpy

from lsst.ts import salobj

import SALPY_ATPtg
import SALPY_ATMCS

In [2]:
import os
print(os.environ["LSST_DDS_DOMAIN"])

lsatmcs


In [3]:
r1 = salobj.Remote(SALPY_ATPtg)
r2 = salobj.Remote(SALPY_ATMCS)

In [ ]:
# r1.evt_summaryState.flush()
await salobj.set_summary_state(r1, salobj.State.STANDBY)

In [ ]:
await salobj.set_summary_state(r1, salobj.State.ENABLED)

In [ ]:
await r2.cmd_start.start(timeout=30)

In [ ]:
await r2.cmd_enable.start(timeout=30)

In [ ]:
await r2.cmd_disable.start(timeout=30)

In [ ]:
await salobj.set_summary_state(r2, salobj.State.STANDBY, timeout=60)

In [ ]:
await salobj.set_summary_state(r2, salobj.State.ENABLED, timeout=60)

In [ ]:
print(salobj.State(r1.evt_summaryState.get().summaryState))
print(salobj.State(r2.evt_summaryState.get().summaryState))

In [ ]:
await salobj.set_summary_state(r2, salobj.State.ENABLED, timeout=300)

In [ ]:
await salobj.set_summary_state(r1, salobj.State.ENABLED, 
                               timeout=300)
# await salobj.set_summary_state(r2, salobj.State.ENABLED, timeout=300)

In [ ]:
print(salobj.State(r1.evt_summaryState.get().summaryState))
print(salobj.State(r2.evt_summaryState.get().summaryState))

In [4]:
location = EarthLocation.from_geodetic(lon=-70.747698*u.deg,
                                                    lat=-30.244728*u.deg,
                                                    height=2663.0*u.m)

In [103]:
time_data = await r1.tel_timeAndDate.next(flush=True, timeout=2)
curr_time_atptg = Time(time_data.tai, format="mjd", scale="tai")
time_err = curr_time_atptg - Time.now()
print(f"Time error={time_err.sec:0.2f} sec")

# Compute RA/Dec for commanded az/el
cmd_elaz = AltAz(alt=70.*u.deg, az=0.*u.deg, 
                 obstime=curr_time_atptg.tai, 
                 location=location)
cmd_radec = cmd_elaz.transform_to(ICRS)

# Compute rot_pa
lst = Angle(time_data.lst, unit=u.hourangle)
lha = lst - cmd_radec.ra
rot_pa = Angle(palpy.pa(lha.radian, cmd_radec.dec.radian, location.lat.radian)*u.radian)
print(f"rotPA: {rot_pa.deg}")

# Start tracking
r1.cmd_raDecTarget.set(
    targetName="atptg_atmcs_integration",
    targetInstance=SALPY_ATPtg.ATPtg_shared_TargetInstances_current,
    frame=SALPY_ATPtg.ATPtg_shared_CoordFrame_icrs,
    epoch=2000,  # should be ignored: no parallax or proper motion
    equinox=2000,  # should be ignored for ICRS
    ra=cmd_radec.ra.hour,
    declination=cmd_radec.dec.deg,
    parallax=0,
    pmRA=0,
    pmDec=0,
    rv=0,
    dRA=0,
    dDec=0,
    rotPA=180.-cmd_elaz.alt.deg,
    rotFrame=SALPY_ATPtg.ATPtg_shared_RotFrame_fixed,
    rotMode=SALPY_ATPtg.ATPtg_shared_RotMode_field,
)
print(f"raDecTarget ra={r1.cmd_raDecTarget.data.ra!r} hour; "
      f"declination={r1.cmd_raDecTarget.data.declination!r} deg")
# script.ataos.cmd_enableCorrection.set(hexapod=True)
# await script.ataos.cmd_enableCorrection.start(timeout=10)



Time error=-0.02 sec
rotPA: 179.32337068010258
raDecTarget ra=16.948365674463098 hour; declination=-10.21629023909037 deg


In [104]:
# r2.evt_target.flush()
# r2.evt_allAxesInPosition.flush()

ack_id = await r1.cmd_raDecTarget.start(timeout=60)
print(f"raDecTarget command result: {ack_id.ack.result}")

# while True:
#     in_position = await r2.evt_allAxesInPosition.next(flush=False)
#     print(f"Got {in_position.inPosition}")
#     if in_position.inPosition:
#         break

raDecTarget command result: Done : OK


In [106]:
ack_id = await r1.cmd_stopTracking.start(timeout=30)

In [108]:
demand = await r1.tel_currentTargetStatus.next(flush=True, timeout=5)
print(demand.demandAz, demand.demandEl, demand.demandRot)
target = await r2.evt_target.next(flush=True, timeout=20)
print(target.azimuth, target.elevation, target.nasmyth1RotatorAngle)

-60:44:44.71 +44:24:28.55 -0:35:31.45
-60.74575255285259 44.40793085156953 -0.5920691487710315


In [101]:
from math import isclose

In [102]:
isclose?

In [14]:
time_data = await r1.tel_timeAndDate.next(flush=True, timeout=2)
curr_time_atptg = Time(time_data.tai, format="mjd", scale="tai")
time_err = curr_time_atptg - Time.now()
print(f"Time error={time_err.sec:0.2f} sec")

# Compute RA/Dec for commanded az/el
cmd_elaz = AltAz(alt=40.*u.deg, az=285.*u.deg, 
                 obstime=curr_time_atptg.tai, 
                 location=location)
cmd_radec = cmd_elaz.transform_to(ICRS)

# Start tracking
r1.cmd_raDecTarget.set(
    targetName="atptg_atmcs_integration",
    targetInstance=SALPY_ATPtg.ATPtg_shared_TargetInstances_current,
    frame=SALPY_ATPtg.ATPtg_shared_CoordFrame_icrs,
    epoch=2000,  # should be ignored: no parallax or proper motion
    equinox=2000,  # should be ignored for ICRS
    ra=cmd_radec.ra.hour,
    declination=cmd_radec.dec.deg,
    parallax=0,
    pmRA=0,
    pmDec=0,
    rv=0,
    dRA=0,
    dDec=0,
    rotPA=-180.,
    rotFrame=SALPY_ATPtg.ATPtg_shared_RotFrame_target,
    rotMode=SALPY_ATPtg.ATPtg_shared_RotMode_field,
)
print(f"raDecTarget ra={r1.cmd_raDecTarget.data.ra!r} hour; "
      f"declination={r1.cmd_raDecTarget.data.declination!r} deg")
# script.ataos.cmd_enableCorrection.set(hexapod=True)
# await script.ataos.cmd_enableCorrection.start(timeout=10)


Time error=-0.03 sec
raDecTarget ra=8.79618671954493 hour; declination=-8.697872837732271 deg


In [ ]:
ack_id = await r2.cmd_startTracking.start(timeout=5, wait_done=False)
print(f"startTracking: {ack_id.ack.ack}::{ack_id.ack.result}")

In [20]:
target = await r2.evt_target.next(flush=True, timeout=5)
print(target.azimuth, target.elevation, target.nasmyth1RotatorAngle)

9.941138816317412 45.018742929232765 26.39707996993859


In [ ]:
demand = await r1.tel_currentTargetStatus.next(flush=True, timeout=5)
demand.demandAz, demand.demandEl, demand.demandRot

In [ ]:
# demand = await r1.evt_trackPosting

In [ ]:
demand = await r1.tel_currentTargetStatus.next(flush=True, timeout=5)
demand.demandAz, demand.demandEl, demand.demandRot

In [30]:
ack_id = await r2.cmd_stopTracking.start(timeout=30)

AckError: msg='Command failed with ack code -301', cmd_id=1559691888, ack=(ack=-301, error=0, result='')

In [ ]:
print(salobj.State(r1.evt_summaryState.get().summaryState))
print(salobj.State(r2.evt_summaryState.get().summaryState))

In [ ]:
def atptg_target_callback(id_data):
    print(id_data.demandAz, id_data.demandEl)

# def atmcs_demand_callback(id_data):
#     print(id_data.)

In [ ]:
r1.tel_currentTargetStatus.callback = atptg_target_callback

In [ ]:
await asyncio.sleep(10)

In [ ]:
r1.tel_currentTargetStatus.callback = None

In [ ]:
demand = await r1.tel_currentTargetStatus.next(flush=True, timeout=5)

In [ ]:
demand = await r1.tel_currentTargetStatus.next(flush=True, timeout=5)
demand.demandAz, demand.demandEl, demand.demandRot

In [ ]:
for i in range(10):
    demand = await r2.evt_target.next(flush=True, timeout=5)
    print(demand.elevation, demand.azimuth)

In [ ]:
ack_id = await r2.cmd_stopTracking.start(timeout=10)

In [ ]:
await salobj.set_summary_state(r1, salobj.State.STANDBY, 
                               timeout=300)
# await salobj.set_summary_state(r2, salobj.State.STANDBY, timeout=300)

In [ ]:
import os

In [ ]:
os.environ["LSST_DDS_DOMAIN"]

In [ ]:
await r1.cmd_exitControl.start()

In [ ]:
50e-3*200

In [ ]:
import palpy

In [ ]:
palpy.pa?

In [ ]:
location = EarthLocation.from_geodetic(lon=-70.747698*u.deg,
                                                    lat=-30.244728*u.deg,
                                                    height=2663.0*u.m)

In [81]:
time_data = await r1.tel_timeAndDate.next(flush=True, timeout=2)
curr_time_atptg = Time(time_data.tai, format="mjd", scale="tai")
time_err = curr_time_atptg - Time.now()
print(f"Time error={time_err.sec:0.2f} sec")

# Compute RA/Dec for commanded az/el
cmd_elaz = AltAz(alt=45.*u.deg, az=0.*u.deg, 
                 obstime=curr_time_atptg.tai, 
                 location=location)
cmd_radec = cmd_elaz.transform_to(ICRS)



Time error=-0.03 sec


In [86]:
cmd_radec.ra.hour,time_data.lst

(12.950322290013087, '12:57:58.6')

In [87]:
lst = Angle(time_data.lst, unit=u.hour)

In [88]:
lha = lst - cmd_radec.ra

In [89]:
lha.hour

0.015955487764689025

In [90]:
rot_pa = Angle(palpy.pa(lha.radian, cmd_radec.dec.radian, location.lat.radian)*u.radian)

In [91]:
rot_pa.deg

179.7081270741601

In [11]:
rot_pa.deg

-10.005029836762048

In [31]:
palpy.pa?